# Object Detection

이미지 내에서 물체의 **위치(Localization)**와 그 **종류(Classfication)**를 찾아내는 것

## 1. Localization

Localization 성능 지표 : IoU(Intersection of Union)

- Label box와의 차를 고려

![image](https://user-images.githubusercontent.com/86637320/136581292-f030b40a-62ea-4935-a4b5-9615877a6c51.png)


Localization label 

- p<sub>c</sub> : 물체가 있을 확률
- b<sub>x</sub>,  b<sub>y</sub> : bbox 좌측 상단 점
- b<sub>h</sub>, b<sub>w</sub> : bbox 폭 정보
- c<sub>1</sub>, c<sub>2</sub>, c<sub>3</sub> : 특정 클래스에 속할 확률

![image](https://user-images.githubusercontent.com/86637320/136582318-5f138210-c0ac-4950-a134-bcf7adc0a2cf.png)


#### Localization 구현 

In [8]:
import tensorflow as tf
from tensorflow import keras

output_num = 1+4+3 # object_prob 1, bbox coord 4, class_prob 3

input_tensor = keras.layers.Input(shape=(224, 224, 3), name='image')
base_model = keras.applications.resnet.ResNet50(
    input_tensor=input_tensor,
    include_top=False,
    weights='imagenet',
    pooling=None,
)
x = base_model.output
x = keras.layers.Conv2D(output_num, 1, 1)(x)
x = keras.layers.Flatten()(x)
preds = keras.layers.Dense(units = 8)(x)

localize_model=keras.Model(inputs=base_model.input, outputs=preds)

localize_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           image[0][0]                      
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

___

## 2. Detection

이미지에서 여러 오브젝트를 찾기 위해선 **sliding window**기법이 활용될 수 있다.

![image](https://user-images.githubusercontent.com/86637320/136590062-10aad3a2-f69a-4558-b1a4-418dcc87f4cc.png)


- object 마다 다른 윈도우 크기를 지정해야하고 한 이미지에 여러 번의 localization을 수행해야하기 때문에 연산량이 너무 많이 요구된다.

👉 연산량과 속도를 개선하기 위해서 **Convolution**을 사용 가능하다.

### Convolution

- Convolution 후 1x1의 결과를 sliding window의 localization의 결과라고 해석하면 Convolution의 receptive field가 결국 window와 같음을 알 수 있다.

- FC layer를 1x1 Conv로 대체한다면(FCN - Fully Convolutional Network) 이미지의 위치, 공간에 관련된 정보를 유지 가능하게 되고 입력 이미지의 크기에도 좌우되지 않게 된다.

만약 **receptive field**에 **여러 물체**가 동시에 검출된다면 어떻게 처리해야 할까?   
: 각 물체마다 다른 크기의 **anchor box**를 가지고 있다고 가정하여 문제를 해결한다.

### Anchor Box

- Anchor box의 설정은 사람이 직접 하기 보다는 k - means 클러스터링을 활용해 가장 일반화에 도움이 되는 anchor box 형태로 설정한다.

- 여러 Anchor box가 검출될 때는 grid cell과의 IoU 값 비교를 통해서 Anchor box를 선택한다.

동일 **receptive field**에 여러 물체가 검출되는 것은 anchor box를 통해서 해결했다. 그럼 한 물체를 여러 receptive field에서 검출한다면 과연 bbox를 어떻게 처리해야 할까? 여러 bbox 중 어떤 것을 최적의 bbox로 설정해야 할까.   
: 겹쳐진 박스 중 **가장 높은 검출 확률**을 가진 박스를 선택한 뒤 이 박스와 *많이 겹치는(IoU 0.5 이상)box는 버려서 설정한다.

___

## 3. 다양한 Object Detection 방법들

![image](https://user-images.githubusercontent.com/86637320/136876852-f71dc7fd-b5df-4dbd-8e45-e80c2abea05f.png)

Many Stage : 물체가 있을 법한 **위치의 후보(proposals)** 들을 뽑아내는 단계, 이후 실제로 물체가 있는지를 Classification과 정확한 바운딩 박스를 구하는 **Regression**을 수행하는 단계가 분리되어 있다.

One stage Detector : 객체의 검출과 분류, 그리고 바운딩 박스 regression을 **한 번에** 하는 방법이다.

### Many Stage

#### (1) R - CNN

![image](https://user-images.githubusercontent.com/86637320/136881156-54fbe87d-5b8e-4c72-9eab-06d1970066e6.png)

R-CNN은 물체가 있을 법한 후보 영역을 뽑아내는 **"Region proposal"** 알고리즘과 후보 영역을 **분류하는 CNN**을 사용한다. 이때 Proposal을 만들어내는 데에는 Selective search라는 비 신경망 알고리즘이 사용된다. 이후에 후보 영역의 Classification과 바운딩 박스의 regression을 위해 신경망을 사용한다. 이때 약 2천개의 이미지를 각각 CNN에 통과시키기 때문에 비효율적이고 느리다는 단점이 있다.

#### (2) Fast R - CNN


![image](https://user-images.githubusercontent.com/86637320/136881480-79990c7a-c2fd-4803-bc86-96ff2c033e25.png)


Fast R-CNN에서는 후보 영역의 classification과 바운딩 박스 regression을 위한 **특성을 한 번에 추출**하여 사용한다. CNN을 통과시킨 후 그 결과를 가지고 잘라내 활용하기 때문에 속도가 훨씬 빨라졌다.

#### (3) Faster R - CNN

![image](https://user-images.githubusercontent.com/86637320/136882057-3e47dfca-58f1-465f-bd4a-085b83536a47.png)

Fast R-CNN은 반복되는 CNN 연산을 크게 줄여냈지만 **region proposal 알고리즘이 병목**이 된다. Faster R-CNN에서는 기존의 Fast R-CNN을 더 빠르게 만들기 위해서 **region proposal 과정에서 RPN(Region Proposal Network)**라고 불리는 신경망 네트워크를 사용한다.

___

### One-Stage

#### (1) YOLO

![image](https://user-images.githubusercontent.com/86637320/136882605-902469f8-979b-4562-8afd-745f2f19c035.png)


YOLO는 이미지를 그리드로 나누고, **Fully Convolutional Network** 연산을 통해 그리드 셀 별로 바운딩 박스를 얻어낸 뒤 **바운딩 박스들에 대해 NMS**를 한 방식이다.

#### (2) SSD

![image](https://user-images.githubusercontent.com/86637320/136886771-82bdc2bc-d45c-492d-876c-b7a366adc11d.png)

**YOLO**의 경우 CNN의 결과로 나온 **Feature Map의 크기가 동일**하기 때문에 다양한 크기의 물체를 감지하기에 적합하지 않다. 이로 인해 작은 물체의 경우 감지가 쉽지 않다. 이를 해결 하기 위해서 다양한 크기의 **Feature Map**으로부터 **Classification과 bbox regression**을 진행한다.